# 🏦 DeepSeek Banking Recovery Agent: Fine-Tuning & Voice Automation

This notebook demonstrates how to build an AI-powered debt collection agent. It covers the full lifecycle:
1.  **Fine-Tuning:** Training `DeepSeek-7b` (via Unsloth) on custom banking intent data (Payment Promises, Refusals, Hardship).
2.  **Inference:** A local text-based chat interface to test the model.
3.  **Real-world Automation:** Integration with **Twilio** (Voice Calls), **AssemblyAI** (Speech-to-Text), and **Google Sheets** (Database) to automate collection calls.

## 🛠️ Step 1: Installation & Setup
Install necessary libraries for Unsloth (LLM training) and the automation pipeline (Twilio, Sheets, Audio).

In [ ]:
%%capture
!pip install unsloth transformers datasets trl torch accelerate
!pip install fastapi uvicorn websockets python-dotenv twilio requests flask assemblyai gspread oauth2client pandas

## 🧠 Step 2: Fine-Tuning DeepSeek with Unsloth
We use `Unsloth` to fine-tune the model 2x faster with 60% less memory. The dataset consists of customer responses labeled with scenarios like "Promise to Pay", "Financial Hardship", etc.

In [ ]:
import json
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

# 1. Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# 2. Load Base Model
print("Loading DeepSeek Model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/deepseek-llm-7b-base",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# 3. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# 4. Define Training Data
def create_clean_training_dataset():
    # Sample of the dataset
    intent_training_data = [
        "<|user|>User says: No, I will not pay.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I’m not going to repay this loan.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I refuse to pay back.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I can’t return the money.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I won’t give you anything.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I’m not paying you back.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: Forget about the repayment.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I don’t intend to return it.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I will not settle this loan.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I won’t pay a single rupee.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I’ll pay today.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I will pay.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Yes, I'll make the payment on time.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I'll pay by the due date, no problem.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Of course, I'll pay on time as usual.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I have the funds ready, will pay on time.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Yes, I'll transfer the money today.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I'll pay without fail on the due date.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Absolutely, I'll pay on time.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I'll ensure payment is made on time.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Yes, I'll pay the full amount on time.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I'll definitely pay by the deadline.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Payment will be made on time, don't worry.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I'll pay as scheduled.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>"
        "<|user|>User says: Money will be sent now.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I can pay tomorrow.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Will pay on 10th.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Payment done today.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I’ll clear it Friday.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I have money, will pay now.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Will pay this week.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Yes, paying soon.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I’ll send cash today.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: i do have the money, i have no problem paying on time<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I will pay on the 15th, please give me some time.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: Can I pay on the 20th instead of the 10th? My salary will be credited by then.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I'm not paying. Why should I?<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I've already paid! You need to check again.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I'm going through a tough time. I can't pay now.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: Can you send me my last payment receipt?<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: Can I negotiate a reduced payment?<|end|>\n<|assistant|>{\"scenario\": \"Debt Settlement\"}<|end|>",
        "<|user|>User says: I promise I'll settle the payment by next week.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I have no money. Why are you calling me?<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: This isn't even my loan. I don't know why you're calling me.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: Can we adjust my payments? I'm really struggling right now.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: I need proof of the charges. Please email it to me.<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: I can only pay 50% of the amount. Will that work?<|end|>\n<|assistant|>{\"scenario\": \"Debt Settlement\"}<|end|>",
        "<|user|>User says: I'll pay tomorrow first thing in the morning.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: This is harassment. Leave me alone.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: Check your records, I paid last month.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I had a medical emergency and can't afford to pay.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: Please send me the full breakdown of what I owe.<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: I'm willing to settle for a lesser amount if you agree.<|end|>\n<|assistant|>{\"scenario\": \"Debt Settlement\"}<|end|>",
        "<|user|>User says: I don't think I owe anything. Check your records.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I can pay half now and the rest next week.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I never took this loan. This is a mistake.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: My business is failing, I need some relief.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: Can you provide the original loan agreement?<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: I can offer you Rs. 5000 to settle this account.<|end|>\n<|assistant|>{\"scenario\": \"Debt Settlement\"}<|end|>",
        "<|user|>User says: I'm not interested in paying this loan.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I want to see the calculation of my outstanding amount.<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: Can we close this with a one-time payment of 70%?<|end|>\n<|assistant|>{\"scenario\": \"Debt Settlement\"}<|end|>",
        "<|user|>User says: I don't have the money right now, but I'll get it to you soon.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I can transfer the money by Thursday evening.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I don't owe this amount. There's been a calculation error.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: My salary got delayed, I'll pay as soon as I receive it.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: Someone else took this loan using my identity.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I made a payment last week. Please verify your system.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I'm facing financial difficulties due to family issues.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: Can you provide the interest calculation sheet?<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: I can pay partially now and settle the rest next month.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I need a detailed statement of all transactions.<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: Is there any way to reduce the interest rate?<|end|>\n<|assistant|>{\"scenario\": \"Debt Settlement\"}<|end|>",
        "<|user|>User says: I refuse to pay until you prove I owe this money.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: This is not my responsibility. Call the right person.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I'm in the hospital and can't deal with this now.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: Please send me the payment history via email.<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>",
        "<|user|>User says: I can only afford to pay in installments going forward.<|end|>\n<|assistant|>{\"scenario\": \"Debt Settlement\"}<|end|>",
        "<|user|>User says: I'll transfer the funds by EOD today.<|end|>\n<|assistant|>{\"scenario\": \"Promise to Pay/ Paying on time\"}<|end|>",
        "<|user|>User says: I don't believe I took this loan. This is fraud.<|end|>\n<|assistant|>{\"scenario\": \"Refusal / Not Paying (Hostile or Calm)\"}<|end|>",
        "<|user|>User says: I'm unemployed and can't make any payments right now.<|end|>\n<|assistant|>{\"scenario\": \"Financial Hardship\"}<|end|>",
        "<|user|>User says: Can you send me the original loan documents?<|end|>\n<|assistant|>{\"scenario\": \"Asking for Proof\"}<|end|>"
    ]
    return Dataset.from_dict({"text": intent_training_data})

dataset = create_clean_training_dataset()

# 5. Train
print("🏋️ Starting Training...")
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=20,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        output_dir="deepseek-bank-bot-model",
    ),
)

trainer.train()
print("Training Completed!")

## 🤖 Step 3: Local Inference (Text Chat)
Before connecting to voice, we test the bot logic locally using a text interface.

In [ ]:
FastLanguageModel.for_inference(model)

def get_user_intent(user_input: str) -> str:
    """Predict intent from user text using the fine-tuned model."""
    prompt = f"User says: {user_input}<|end|>\n<|assistant|>"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_new_tokens=150,
        use_cache=True,
        temperature=0.1,
        pad_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract JSON intent
    import re
    json_pattern = r'\{[^}]*"scenario"[^}]*\}'
    matches = re.findall(json_pattern, response)
    if matches:
        return json.loads(matches[0]).get("scenario", "Unknown")
    return "Unknown"

# Test the function
test_input = "I will definitely pay the amount by next Friday."
print(f"Input: {test_input}")
print(f"Predicted Intent: {get_user_intent(test_input)}")

## 📞 Step 4: Voice Automation (Twilio + AssemblyAI)
This section connects the fine-tuned model to the real world.
1.  **Read** customer list from Google Sheets.
2.  **Call** customer via Twilio.
3.  **Record & Transcribe** response via AssemblyAI.
4.  **Analyze** intent using our DeepSeek model.
5.  **Update** Google Sheet with the result.

In [ ]:
import os
import time
import requests
import pandas as pd
import gspread
import assemblyai as aai
from twilio.rest import Client
from oauth2client.service_account import ServiceAccountCredentials
from requests.auth import HTTPBasicAuth

# CONFIGURATION (FILL THESE)
TWILIO_SID = "YOUR_TWILIO_ACCOUNT_SID"
TWILIO_AUTH_TOKEN = "YOUR_TWILIO_AUTH_TOKEN"
TWILIO_FROM_NUMBER = "+1234567890"  # Your Twilio number
ASSEMBLYAI_KEY = "YOUR_ASSEMBLYAI_API_KEY"
GOOGLE_CREDS_FILE = "credentials.json" # Upload your service account json to Colab
GOOGLE_SHEET_URL = "YOUR_GOOGLE_SHEET_URL"

# Initialize Clients
client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
aai.settings.api_key = ASSEMBLYAI_KEY

# Setup Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_CREDS_FILE, scope)
gc = gspread.authorize(creds)
worksheet = gc.open_by_url(GOOGLE_SHEET_URL).sheet1

In [ ]:
def make_call_and_record(name, emi_date, to_number):
    """Initiates a call, plays a message, and returns the recording URL"""
    
    speak_text = f"Hello {name}, this is a call regarding your EMI due on {emi_date}. Will you be able to pay on time?"
    
    # Twilio Voice Response (TwiML)
    twiml = f'''<Response>
        <Say voice="Google.en-IN-Standard-E">{speak_text}</Say>
        <Record timeout="5" maxLength="30" playBeep="true"/>
        <Say>Thank you. Goodbye.</Say>
    </Response>'''
    
    print(f"Calling {name}...")
    call = client.calls.create(
        twiml=twiml,
        to=to_number,
        from_=TWILIO_FROM_NUMBER,
        record=True
    )
    
    # Wait for completion (Simple Polling)
    while call.status not in ['completed', 'failed', 'busy', 'no-answer']:
        call = client.calls(call.sid).fetch()
        time.sleep(2)
    
    # Fetch Recordings
    recordings = client.recordings.list(call_sid=call.sid)
    if recordings:
        rec_url = f"https://api.twilio.com/2010-04-01/Accounts/{TWILIO_SID}/Recordings/{recordings[0].sid}.wav"
        print("Recording found!")
        return rec_url
    return None

def download_audio(url, filename="rec.wav"):
    """Downloads the audio file from Twilio"""
    resp = requests.get(url, auth=HTTPBasicAuth(TWILIO_SID, TWILIO_AUTH_TOKEN))
    with open(filename, 'wb') as f:
        f.write(resp.content)
    return filename

def transcribe_audio(filename):
    """Transcribes audio using AssemblyAI"""
    transcriber = aai.Transcriber()
    transcript = transcriber.transcribe(filename)
    return transcript.text

## 🚀 Step 5: Run the Automation Loop
This loop iterates through your Google Sheet, makes calls, processes the audio, determines the intent using the AI, and updates the sheet.

In [ ]:
# Fetch Data
data = worksheet.get_all_records()
df = pd.DataFrame(data)

print(f"Found {len(df)} customers to contact.")

for index, row in df.iterrows():
    name = row['Name']      # Ensure your sheet has 'Name' column
    date = row['Date']      # Ensure your sheet has 'Date' column
    number = str(row['Phone']) # Ensure your sheet has 'Phone' column
    
    # 1. Make Call
    rec_url = make_call_and_record(name, date, number)
    
    if rec_url:
        # 2. Download & Transcribe
        audio_file = download_audio(rec_url)
        customer_text = transcribe_audio(audio_file)
        print(f"Customer said: {customer_text}")
        
        # 3. Analyze Intent with DeepSeek
        intent = get_user_intent(customer_text)
        print(f"Detected Intent: {intent}")
        
        # 4. Update Sheet (Assuming column 4 is for Status)
        # Row index in Sheets is 1-based, plus header row
        worksheet.update_cell(index + 2, 4, intent)
    else:
        print("Call failed or no recording.")
        worksheet.update_cell(index + 2, 4, "Call Failed")
    
    # Rate limit pause
    time.sleep(2)